In [1]:
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/00226/OpportunityUCIDataset.zip -P ../data/raw/

'wget' 不是内部或外部命令，也不是可运行的程序
或批处理文件。


In [2]:
!mkdir ../data/processed
!python3 ../src/data/preprocess_data.py -i  ../data/raw/OpportunityUCIDataset.zip -o ../processed/oppChallenge_gestures.data

SyntaxError: invalid syntax (<ipython-input-2-05c2b0460174>, line 1)

### Setup

In [27]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import _pickle as cp
import matplotlib.pyplot as plt
import pandas as pd
import sklearn.metrics as metrics
from sklearn.metrics import plot_confusion_matrix

import torch
from torch import nn
import torch.nn.functional as F

from sklearn.metrics import confusion_matrix
import matplotlib
matplotlib.use('TkAgg')
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report

#NB_SENSOR_CHANNELS = 113
NB_SENSOR_CHANNELS = 15
SLIDING_WINDOW_LENGTH = 50
SLIDING_WINDOW_STEP = 25
NB_CLASS = 4

In [28]:
import os
import sys

# add the 'src' directory as one where we can import modules
src_dir = os.path.join(os.getcwd(), os.pardir, 'src')
sys.path.append(src_dir)

from data.sliding_window import sliding_window

## Load in Data

In [29]:

def load_dataset(filename):
    
    with open(filename, 'rb') as f:
        data = cp.load(f)
    
    X_train, y_train = data[0]
    X_test, y_test = data[1]

    print(" ..from file {}".format(filename))
    print(" ..reading instances: train {0}, test {1}".format(X_train.shape, X_test.shape))

    X_train = X_train.astype(np.float32)
    X_test = X_test.astype(np.float32)
    
    print("..part of x_train",type(X_train))
    #print(X_train[:23,:2])
    # The targets are casted to int8 for GPU compatibility.
    y_train = y_train.astype(np.uint8)
    y_test = y_test.astype(np.uint8)

    return X_train, y_train, X_test, y_test

print("Loading data...")
#X_train, y_train, X_test, y_test = load_dataset('../data/processed/oppChallenge_gestures.data')
X_train, y_train, X_test, y_test = load_dataset('C:\\Users\\Zhiha\\Documents\\file_only\\research\\tool_program\\ori\\Sensor-Based-Human-Activity-Recognition-DeepConvLSTM-Pytorch\\data\\processed\\mygestures_ww.data')


Loading data...
 ..from file C:\Users\Zhiha\Documents\file_only\research\tool_program\ori\Sensor-Based-Human-Activity-Recognition-DeepConvLSTM-Pytorch\data\processed\mygestures_ww.data
 ..reading instances: train (60763, 15), test (17422, 15)
..part of x_train <class 'numpy.ndarray'>


## Data Segmentation

In [30]:
assert NB_SENSOR_CHANNELS == X_train.shape[1]
def opp_sliding_window(data_x, data_y, ws, ss):
    data_x = sliding_window(data_x, (ws, data_x.shape[1]), (ss, 1))
    data_y = np.asarray([[i[-1]] for i in sliding_window(data_y, ws, ss)])
    return data_x.astype(np.float32), data_y.reshape(len(data_y)).astype(np.uint8)

# Sensor data is segmented using a sliding window mechanism
X_train, y_train = opp_sliding_window(X_train, y_train, SLIDING_WINDOW_LENGTH, SLIDING_WINDOW_STEP)
X_test, y_test = opp_sliding_window(X_test, y_test, SLIDING_WINDOW_LENGTH, SLIDING_WINDOW_STEP)
#print(X_train[:24,:3])
print(X_train.shape)
# Data is reshaped
X_train = X_train.reshape((-1, SLIDING_WINDOW_LENGTH, NB_SENSOR_CHANNELS)) # for input to Conv1D
X_test = X_test.reshape((-1, SLIDING_WINDOW_LENGTH, NB_SENSOR_CHANNELS)) # for input to Conv1D

#print(y_train[10:25,])
print(" ..after sliding and reshaping, train data: inputs {0}, targets {1}".format(X_train.shape, y_train.shape))
print(" ..after sliding and reshaping, test data : inputs {0}, targets {1}".format(X_test.shape, y_test.shape))

(2429, 50, 15)
 ..after sliding and reshaping, train data: inputs (2429, 50, 15), targets (2429,)
 ..after sliding and reshaping, test data : inputs (695, 50, 15), targets (695,)


## Get labels

In [31]:
#='D:\\test\\project2files\\2\\files\\labels.csv'
def return_labels(f,series):
    ss = []
    df = pd.read_csv(f,header=None)
    state = df.iloc[1:,0].to_numpy()
    activities = df.iloc[0,1:].to_numpy()
    state_len = len(state)
    act_len = len(activities)
    for s in series:        
        x = s % act_len #acts
        y = s // act_len #states
        #print(state[y]+'+'+activities[x])
        ss.append(str(state[y])+'+'+str(activities[x]))
    return ss

## draw confusion matrix

In [32]:
tick_marks = np.array(range(NB_CLASS)) + 0.5


def plot_confusion_matrix(cm, title='Confusion Matrix', cmap=plt.cm.binary):
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    xlocations = np.array(range(NB_CLASS))
    plt.xticks(xlocations, xlocations, rotation=90)
    plt.yticks(xlocations, xlocations)
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    

    
def print_cm(cm,epoch):
    np.set_printoptions(precision=2)
    cm_normalized = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
    print (cm_normalized)
    plt.figure(figsize=(12, 8), dpi=120)

    ind_array = np.arange(NB_CLASS)
    x, y = np.meshgrid(ind_array, ind_array)

    for x_val, y_val in zip(x.flatten(), y.flatten()):
        c = cm_normalized[y_val][x_val]
        if c > 0.01:
            plt.text(x_val, y_val, "%0.2f" % (c,), color='red', fontsize=7, va='center', ha='center')
    # offset the tick
    plt.gca().set_xticks(tick_marks, minor=True)
    plt.gca().set_yticks(tick_marks, minor=True)
    plt.gca().xaxis.set_ticks_position('none')
    plt.gca().yaxis.set_ticks_position('none')
    plt.grid(True, which='minor', linestyle='-')
    plt.gcf().subplots_adjust(bottom=0.15)

    plot_confusion_matrix(cm_normalized, title='Normalized confusion matrix')
    # show confusion matrix
    plt.savefig(os.path.abspath(os.path.join(os.getcwd(), ".."))+ '/cm_results/'+'t_matrix'+str(epoch)+'.png', format='png')
    plt.close()
    #provide report


## Define the Model

In [33]:
class HARModel(nn.Module):
    
    def __init__(self, n_hidden=128, n_layers=1, n_filters=64, 
                 n_classes=NB_CLASS, filter_size=3, drop_prob=0.5):
        super(HARModel, self).__init__()
        self.drop_prob = drop_prob
        self.n_layers = n_layers
        self.n_hidden = n_hidden
        self.n_filters = n_filters
        self.n_classes = n_classes
        self.filter_size = filter_size
             
        self.conv1 = nn.Conv1d(NB_SENSOR_CHANNELS, n_filters, filter_size)
        self.conv2 = nn.Conv1d(n_filters, n_filters, filter_size)
        self.conv3 = nn.Conv1d(n_filters, n_filters, filter_size)
        self.conv4 = nn.Conv1d(n_filters, n_filters, filter_size)
        
        self.lstm1  = nn.LSTM(n_filters, n_hidden, n_layers)
        self.lstm2  = nn.LSTM(n_hidden, n_hidden, n_layers)
        
        self.fc = nn.Linear(n_hidden, n_classes)

        self.dropout = nn.Dropout(drop_prob)
    
    def forward(self, x, hidden, batch_size):
        #print(x.size())
        
        #x = x.view(-1, NB_SENSOR_CHANNELS, SLIDING_WINDOW_LENGTH)#reshape
        x = x.reshape(-1, NB_SENSOR_CHANNELS, SLIDING_WINDOW_LENGTH)#reshape
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = F.relu(self.conv3(x))
        x = F.relu(self.conv4(x))
        m = x.size()[2]
        x = x.view(m, -1, self.n_filters)
        x, hidden = self.lstm1(x, hidden)
        x, hidden = self.lstm2(x, hidden)
        
        x = x.contiguous().view(-1, self.n_hidden)
        x = self.dropout(x)
        x = self.fc(x)
        
        out = x.view(batch_size, -1, self.n_classes)[:,-1,:]
        
        return out, hidden
    
    def init_hidden(self, batch_size):
        ''' Initializes hidden state '''
        # Create two new tensors with sizes n_layers x batch_size x n_hidden,
        # initialized to zero, for hidden state and cell state of LSTM
        weight = next(self.parameters()).data
        '''
        if (train_on_gpu):
            hidden = (weight.new(self.n_layers, batch_size, self.n_hidden).zero_().cuda(),
                  weight.new(self.n_layers, batch_size, self.n_hidden).zero_().cuda())
        else:
            hidden = (weight.new(self.n_layers, batch_size, self.n_hidden).zero_(),
                      weight.new(self.n_layers, batch_size, self.n_hidden).zero_())
        '''
        hidden = (weight.new(self.n_layers, batch_size, self.n_hidden).zero_(),
                      weight.new(self.n_layers, batch_size, self.n_hidden).zero_())
        
        return hidden
    
net = HARModel()

## Initialize Model Weights

In [34]:
def init_weights(m):
    if type(m) == nn.LSTM:
        for name, param in m.named_parameters():
            if 'weight_ih' in name:
                torch.nn.init.orthogonal_(param.data)
            elif 'weight_hh' in name:
                torch.nn.init.orthogonal_(param.data)
            elif 'bias' in name:
                param.data.fill_(0)
    elif type(m) == nn.Conv1d or type(m) == nn.Linear:
        torch.nn.init.orthogonal_(m.weight)
        m.bias.data.fill_(0)
net.apply(init_weights)    

HARModel(
  (conv1): Conv1d(15, 64, kernel_size=(3,), stride=(1,))
  (conv2): Conv1d(64, 64, kernel_size=(3,), stride=(1,))
  (conv3): Conv1d(64, 64, kernel_size=(3,), stride=(1,))
  (conv4): Conv1d(64, 64, kernel_size=(3,), stride=(1,))
  (lstm1): LSTM(64, 128)
  (lstm2): LSTM(128, 128)
  (fc): Linear(in_features=128, out_features=4, bias=True)
  (dropout): Dropout(p=0.5, inplace=False)
)

## Generate Minibatches

In [35]:
def iterate_minibatches(inputs, targets, batchsize, shuffle=True):
    assert len(inputs) == len(targets)
    if shuffle:
        indices = np.arange(len(inputs))
        np.random.shuffle(indices)
    for start_idx in range(0, len(inputs) - batchsize + 1, batchsize):
        if shuffle:
            excerpt = indices[start_idx:start_idx + batchsize]
        else:
            excerpt = slice(start_idx, start_idx + batchsize)
        yield inputs[excerpt], targets[excerpt]

In [36]:
## check if GPU is available
#train_on_gpu = torch.cuda.is_available()
#if(train_on_gpu):
    #print('Training on GPU!')
#else: 
    #print('No GPU available, training on CPU; consider making n_epochs very small.')

## Train the Network

In [37]:

def train(net, epochs=50, batch_size=10, lr=0.01):
    min_loss = 100
    max_acc = 0.1
    opt = torch.optim.SGD(net.parameters(), lr=lr, momentum=0.9, weight_decay=1e-4)
    criterion = nn.CrossEntropyLoss()
    
    #if(train_on_gpu):
        #net.cuda()
    valloss =[]
    valacc = []
    traloss =[]
    t = np.arange(NB_CLASS)
    y_unique = return_labels('D:\\test\\project2files\\2\\test\\files\\labels.csv',t)
    for e in range(epochs):
        
        # initialize hidden state
        h = net.init_hidden(batch_size)         
        train_losses = []    
        net.train()
        for batch in iterate_minibatches(X_train, y_train, batch_size):
            x, y = batch

            inputs, targets = torch.from_numpy(x), torch.from_numpy(y)
            

            #if(train_on_gpu):
            #        inputs, targets = inputs.cuda(), targets.cuda()

            # Creating new variables for the hidden state, otherwise
            # we'd backprop through the entire training history
            h = tuple([each.data for each in h])
            
            # zero accumulated gradients
            opt.zero_grad()   
            
            # get the output from the model
            output, h = net(inputs, h, batch_size)
            
            loss = criterion(output, targets.long())
            train_losses.append(loss.item())
            loss.backward()
            opt.step()
            
        val_h = net.init_hidden(batch_size)
        val_losses = []
        ##
        val_output = []
        val_target = []
        ##
        accuracy=0
        f1score=0
        net.eval()
        with torch.no_grad():
            for batch in iterate_minibatches(X_test, y_test, batch_size):
                x, y = batch     
                val_target.extend(y)
                inputs, targets = torch.from_numpy(x), torch.from_numpy(y)
                #print("targets: \n",targets)
                #val_target.extend(targets)
                

                val_h = tuple([each.data for each in val_h])

                #if(train_on_gpu):
                #    inputs, targets = inputs.cuda(), targets.cuda()
                    
                output, val_h= net(inputs, val_h, batch_size)
                #print("output: \n",output)
                #val_output.extend(output)

                val_loss = criterion(output, targets.long())
                val_losses.append(val_loss.item())

                top_p, top_class = output.topk(1, dim=1)#get the possible class prediction
                
                equals = top_class == targets.view(*top_class.shape).long()
                #print("top_class: \n",top_class.view(*targets.shape))
                val_output.extend(top_class.view(*targets.shape))
                accuracy += torch.mean(equals.type(torch.FloatTensor))
                f1score += metrics.f1_score(top_class.cpu(), targets.view(*top_class.shape).long().cpu(), average='weighted')
            
        net.train() # reset to train mode after iterationg through validation data
        print(batch_size)        
        print("Epoch: {}/{}...".format(e+1, epochs),
        "Train Loss: {:.4f}...".format(np.mean(train_losses)),
        "Val Loss: {:.4f}...".format(np.mean(val_losses)),
        "Val Acc: {:.4f}...".format(accuracy/(len(X_test)//batch_size)),
        "F1-Score: {:.4f}...".format(f1score/(len(X_test)//batch_size)))
        print("Length of true and predict:{}/{}...".format(len(val_target),len(val_output)))
        cm = confusion_matrix(val_target,val_output)
        print(classification_report(val_target,val_output,target_names = np.array(y_unique)))
        
        tl = np.mean(train_losses)
        vl = np.mean(val_losses)
        ac = accuracy/(len(X_test)//batch_size)
        if vl < min_loss or ac>max_acc:
            min_loss = vl
            max_acc = ac
            print("save model")
            torch.save(net.state_dict(),os.path.abspath(os.path.join(os.getcwd(), "..","cm_results","model"+".pth")))
            print_cm(cm,e)
        valloss.append(vl)
        valacc.append(ac)
        traloss.append(tl)
    x = np.arange(epochs)
    
    plt.gca()
    plt.plot(x,valloss,color='green',label='val_loss')
    plt.plot(x,traloss,color='red',label='train_loss')
    plt.plot(x,valacc,color='blue',label='val_acc')
    plt.xlabel('epochs')
    plt.savefig(os.path.abspath(os.path.join(os.getcwd(), "..","cm_results","result_t.jpg")))
    plt.close()
train(net)

10
Epoch: 1/50... Train Loss: 1.2764... Val Loss: 1.2950... Val Acc: 0.3449... F1-Score: 0.4952...
Length of true and predict:690/690...
              precision    recall  f1-score   support

standing+nan       0.00      0.00      0.00       259
 standing+up       0.00      0.00      0.00        94
 walking+nan       0.34      1.00      0.51       238
  walking+up       0.00      0.00      0.00        99

    accuracy                           0.34       690
   macro avg       0.09      0.25      0.13       690
weighted avg       0.12      0.34      0.18       690

save model
[[0. 0. 1. 0.]
 [0. 0. 1. 0.]
 [0. 0. 1. 0.]
 [0. 0. 1. 0.]]
10
Epoch: 2/50... Train Loss: 1.2663... Val Loss: 1.3052... Val Acc: 0.3377... F1-Score: 0.4793...
Length of true and predict:690/690...
              precision    recall  f1-score   support

standing+nan       0.00      0.00      0.00       261
 standing+up       0.00      0.00      0.00        95
 walking+nan       0.34      1.00      0.50       233
  

10
Epoch: 14/50... Train Loss: 0.2471... Val Loss: 0.2441... Val Acc: 0.9203... F1-Score: 0.9219...
Length of true and predict:690/690...
              precision    recall  f1-score   support

standing+nan       0.96      0.97      0.97       258
 standing+up       0.76      0.95      0.84        94
 walking+nan       0.97      0.96      0.97       237
  walking+up       0.87      0.67      0.76       101

    accuracy                           0.92       690
   macro avg       0.89      0.89      0.88       690
weighted avg       0.92      0.92      0.92       690

save model
[[0.97 0.   0.01 0.02]
 [0.   0.95 0.   0.05]
 [0.04 0.   0.96 0.  ]
 [0.01 0.28 0.04 0.67]]
10
Epoch: 15/50... Train Loss: 0.2203... Val Loss: 0.2289... Val Acc: 0.9101... F1-Score: 0.9131...
Length of true and predict:690/690...
              precision    recall  f1-score   support

standing+nan       0.97      0.98      0.97       259
 standing+up       0.69      0.95      0.80        95
 walking+nan       0.9

10
Epoch: 28/50... Train Loss: 0.1021... Val Loss: 0.3095... Val Acc: 0.8986... F1-Score: 0.8920...
Length of true and predict:690/690...
              precision    recall  f1-score   support

standing+nan       0.99      0.92      0.95       259
 standing+up       0.82      0.73      0.78        94
 walking+nan       0.95      0.97      0.96       238
  walking+up       0.67      0.84      0.74        99

    accuracy                           0.90       690
   macro avg       0.86      0.86      0.86       690
weighted avg       0.91      0.90      0.90       690

10
Epoch: 29/50... Train Loss: 0.0977... Val Loss: 0.2616... Val Acc: 0.8971... F1-Score: 0.8999...
Length of true and predict:690/690...
              precision    recall  f1-score   support

standing+nan       0.97      0.98      0.97       261
 standing+up       0.75      0.69      0.72        93
 walking+nan       0.95      0.97      0.96       237
  walking+up       0.70      0.72      0.71        99

    accuracy     

10
Epoch: 43/50... Train Loss: 0.0454... Val Loss: 0.3382... Val Acc: 0.9072... F1-Score: 0.9106...
Length of true and predict:690/690...
              precision    recall  f1-score   support

standing+nan       0.96      0.96      0.96       260
 standing+up       0.87      0.71      0.78        95
 walking+nan       0.94      0.96      0.95       236
  walking+up       0.73      0.84      0.78        99

    accuracy                           0.91       690
   macro avg       0.88      0.87      0.87       690
weighted avg       0.91      0.91      0.91       690

10
Epoch: 44/50... Train Loss: 0.0440... Val Loss: 0.2898... Val Acc: 0.9087... F1-Score: 0.9093...
Length of true and predict:690/690...
              precision    recall  f1-score   support

standing+nan       0.96      0.95      0.96       259
 standing+up       0.79      0.81      0.80        95
 walking+nan       0.96      0.96      0.96       236
  walking+up       0.76      0.76      0.76       100

    accuracy     